# Part 1: Extract key phrases

Importing required packages

In [ ]:
import nltk
import gensim
import warnings,pickle
warnings.filterwarnings('ignore')

In [ ]:
import sklearn, pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop=stopwords.words('english')

Importing data and analyzing

In [ ]:
df_pd=pd.read_csv(r'')

In [ ]:
df_pd['comment_orig']=df_pd['comment']
df_pd.columns

In [ ]:
df_case_grp=df_pd[['case_type','i_customer_id']].groupby(['case_type']).count().sort_values(['i_customer_id'],asc)
df_case_grp.loc["Debit card Reissuance"].values[0]

In [ ]:
df_pd=df_pd[df_pd.case_type.isin(["Debit card Reissuance","Transaction Related","Online transaction status","Debit Card Usage"])]
print(len(df_pd))

In [ ]:
df_pd=df_pd[df_pd['comment'].str.len()>5]
print(len(df_pd))

Expanding abbreviated test in comment

In [1]:
def abbreviate_text(inp_df):
    inp_df.comment=inp_df.comment.str.replace(r'a/c',"account")
    inp_df.comment=inp_df.comment.str.replace(r'\b(ac)\b',"account")
    inp_df.comment=inp_df.comment.str.replace(r'\b(acc)\b',"account")
    inp_df.comment=inp_df.comment.str.replace(r'\b(cm)\b',"customer")
    inp_df.comment=inp_df.comment.str.replace(r'\b(cus)\b',"customer")
    inp_df.comment=inp_df.comment.str.replace(r'\b(cust)\b',"customer")
    inp_df.comment=inp_df.comment.str.replace(r'\b(d/c)\b',"debit card")
    inp_df.comment=inp_df.comment.str.replace(r'\b(dc)\b',"debit card")
    inp_df.comment=inp_df.comment.str.replace(r'\b(txn)\b',"transaction")
    inp_df.comment=inp_df.comment.str.replace(r'\b(trxn)\b',"transaction")
    inp_df.comment=inp_df.comment.str.replace(r'\b(amt)\b',"amount")
    inp_df.comment=inp_df.comment.str.replace(r'\b(amnt)\b',"amount")
    inp_df.comment=inp_df.comment.str.replace(r'\b(plz)\b',"please")
    inp_df.comment=inp_df.comment.str.replace(r'\b(fd)\b',"fixed deposit")
    inp_df.comment=inp_df.comment.str.replace(r'\b(rnew)\b',"renew")
    inp_df.comment=inp_df.comment.str.replace(r'\b(sup)\b',"supervisor")
    inp_df.comment=inp_df.comment.str.replace(r'\b(atd)\b',"atm withdrawal")
    return inp_df

Convert comment to lower case and call abbreviate fn

In [ ]:
df_pd['comment']=df_pd.comment.str.lower()
df_pd=abbreviate_text(df_pd)
pd.set_option('display.max_colwidth',-1)
df_pd['comment'].head()

In [ ]:
df_pd['comment']=df_pd.comment.str.replace('imps/na/xxx/rrn:/pa/'," ")
df_pd['comment']=df_pd.comment.str.replace('\W'," ")
df_pd['comment']=df_pd.comment.str.replace('[\s+]',"",regex=False)

In [ ]:
domain_stop=['please','kindly','check','dear','still','regards','customer']
stop.extend(domain_stop)

Removing stop words from the data

In [ ]:
def clean_data(inp_df):
    clean_data_lst=[]
    for i,row in inp_df.iterrows():
        tokens=str(row['comment']).split()
        tokens=[i for i in tokens if not i in stop]
        clean_data_lst.append(' '.join(word for word in tokens))
    return clean_data_lst

Remove text with fewer inputs

In [ ]:
df_pd=df_pd[df_pd['comment'].str.len()>5]
print(len(df_pd))

In [ ]:
pd.set_option('display.max_colwidth',-1)
df_pd['comment'].head()

Defining stopwords for RAKE algorithm

In [ ]:
stop_RAKE=stop[:]
stop_RAKE.remove("didn't")
stop_RAKE.remove("not")

In [ ]:
import RAKE
Rake=RAKE.Rake(stop_RAKE)

Extracting noun phrase

In [ ]:
def get_noun_phrase(inp_comment):
    NP="NP: {(<V\w+>|<NN\w?>+.*<NN\w?>)}"
    chnkr=nltk.RegexpParser(NP)
    current_chunk=[]
    cont_chunk=[]
    tokens=word_tokenize(str(inp_comment))
    tokens=[i for i in tokens if not i in stop]
    text_chunk=chnkr.parse(nltk.pos_tag(tokens))
    
    for subtree in text_chunk:
        if type(subtree)==nltk.Tree:
            current_chunk.append(' '.join([word for word, POS in subtree.leaves()]))
        elif current_chunk:
            ne= ' '.join(current_chunk)
            if ne not in cont_chunk:
                cont_chunk.append(ne)
                cont_chunk=[]
    return ', '.join(cont_chunk)

Remove duplicate key words from the noun phrase

In [ ]:
def rem_dup_ele(inp_lst,inp_comment):
    result=[]
    for string in inp_lst:
        if not any([string in r for r in inp_lst if string!=r])
            result.append(string)
            
    if len(result)<=6:
        return result
    else:
        new_result=result[0:5]
        result=result[5:]
        noun_phrase=get_noun_phrase(inp_comment)
        for phrase in result:
            if any(phrase in np for np in noun_phrase):
                new_result.append(phrase)
        return new_result

Extract Key Phrase

In [ ]:
def get_candidate_key(inp_df):
    inp_df['key_phrase']=''
    for i,row in inp_df.iterrows():
        cand_key=Rake.run(str(row.comment))
        keys=', '.join(key[0] for key in cand_key)
        keys_lst=keys.split(",")
        if len(key_lst)>8:
            inp_df.loc[i,'key_phrase']=', '.join(rem_dup_ele(keys_lst,str(row.comment)))
        else:
            inp_df.loc[i,'key_phrase']=keys
        return inp_df    

In [ ]:
inp_df=get_candidate_key(inp_df)
pd.set_option('display.max_colwidth',-1)
inp_df.iloc[:,-2:]

# Case 2: Ticket case classifier

In [ ]:
def clean_data(inp_df):
    clean_data_lst=[]
    for i,row in inp_df.iterrows():
        tokens=str(row['comment']).split()
        tokens=[i for i in tokens if not i in stop]
        clean_data_lst.append(' '.join(word for word in tokens))
    return clean_data_lst

In [ ]:
df_pd['comment_token']=clean_data(df_pd)
df_pd['comment_token'].head()

Grouping and analysing Ticket case types

In [ ]:
df_grp=df_pd[['case_type','i_customer_id']].groupby(['case_type']).count().sort_values(['i_customer_id'],ascending=False)

In [ ]:
df_pd.isna().sum()

Train test split

In [3]:
from sklearn.model_selection import train_test_split,cross_val_score
X=df_pd['comment']
y=df_pd['case_type']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)
len(df_pd)

C:\Users\anand\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


Vectorizing the input data for ml model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer(max_features=1000,ngram_range=(1,3)).fit(X_train)
train_vect=vect.transform(X_train)
test_vect=vect.transform(X_test)
with open('RF_Vect.pkl','wb')
    pickle.dump(vect,file)

Defining model for training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
np.array(df_pd.case_type)

In [ ]:
clf=RandomForestClassifier(n_estimators=50,min_samples_leaf=25)
clf.fit(train_vect,y_train)
predict=clf.predict(test_vect)

In [ ]:
with open('RF_Model_new.pkl','wb')
    pickle.dump(clf,file)

In [ ]:
print("Model Train Score: ",clf.score(train_vect,y_train).round(2))
print("Model Test Score: ",clf.score(test_vect,y_test).round(2))

In [ ]:
X_pred=pd.DataFfame(X_test).reset_index(drop=True)
X_pred['Predicted Case']=pd.Series(predict)
X_pred['actual Case']=y_test.rest_index(drop=True)

In [ ]:
category=np.unique(y_train)
category

Printing confusion matrix for Ticket case types

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predict,labels=np.unique(y_train))
print(cm)
i=0
for row in cm:
    print(category[i]+" : "+str(df_case_grp.loc[category[i]].values[0])+" : "+str(row[i]*100/sum(row)))
    i=i+1

In [ ]:
df_pd_relevent=df_pd[df_pd.case_type.isin(["case related","Debit Card Usage","Online transmission Status","Debit Card Reissued"])]
len(df_pd_relevent)

In [ ]:
X_test_vect=vect.transform(df_pd_relevent['comment'])
df_pd_relevent=df_pd_relevent{"case_type"}
print("Model Test Relevent Score: ",clf.score(X_test_vect_relevent,Y_test_relevent).round(2))

In [ ]:
X_pred[X_pred["actual case"]=="Supervisor Call back"].iloc[:30]

In [ ]:
X_pred[X_pred["actual case"]=="Foreclosure of Loan"].iloc[:30]

In [ ]:
X_pred.iloc[:30]

# Part 3: Prioratize Tickets

Importing the pretraining sentiment analyzer model

In [ ]:
from nltk.sentiment.vadar import SentimentIntensityAnalyzer
sent=SentimentIntensityAnalyzer()

Score priority based on ticket sentiment

In [ ]:
def get_polarity_score(text):
    ss=sent.polarity_score(text)
    score=0
    if ss['compound']>0.5:
        score=1
    elif ss['compound']>0.0:
        score=2
    elif ss['compound']>-0.4:
        score=3
    elif ss['compound']>-0.7:
        score=4
    else:
        score=5
    return score

In [ ]:
def get_priority_score(inp_df):
    inp_df['priority_score']=''
    for i,row in inp_df.iterrows():
        inp_df.loc[i,'priority_score']=get_priority_keywords(row)+get_priority_score(str(row.comment))
    return inp_df

Prioratize tickets and view the results

In [ ]:
inp_df=get_priority_score(df_pd.head(50))
pd.set_option('display.max_colwidth',-1)
inp_df.iloc[:,-3:]